### Using the basic preprocessing module of `s2spy`

To start we need to import the `preprocess` module, and load in some example data (more details about the example data can be found [here](https://github.com/AI4S2S/cookbook/tree/main/data)).


In [1]:
import urllib
import xarray as xr
from s2spy import preprocess
import matplotlib.pyplot as plt

In [2]:
# URL of the dataset from zenodo
sst_url = "https://zenodo.org/record/8186914/files/sst_daily_1959-2021_5deg_Pacific_175_240E_25_50N.nc"
sst_field = "sst_daily_1959-2021_5deg_Pacific_175_240E_25_50N.nc"
urllib.request.urlretrieve(sst_url, sst_field)

data = xr.open_dataset(sst_field)

The preprocessor is instatiated with the basic settings, the size of the rolling window, the detrending method, and if climatology should be removed from the data.

In [ ]:
preprocessor = preprocess.Preprocessor(
    rolling_window_size=25,
    detrend="linear",
    subtract_climatology=True,
)

The preprocesssor can be fit to data, which updates the preprocessor object in place, storing the trend and climatology:

In [ ]:
preprocessor.fit(data)

In [ ]:
preprocessor.trend

Now the climatology and trend will be stored in `preprocessor`. 

In the example case the climatology of one location is stored as follows:

In [ ]:
preprocessor.climatology["sst"].isel(latitude=0, longitude=0).plot()

The trend has multiple components, depending on the method. The "linear" method has a slope and an intercept. (Currently, only linear detrending is implemented). 

The example data has already been detrended, so its slope is (basically) 0:

In [ ]:
preprocessor.trend["slope"].sst.isel(latitude=0, longitude=0).values

To apply the detrending and climatology removal to a dataset, the `transform` method is used:

In [ ]:
preprocessed_data = preprocessor.transform(data)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8, 6))
data.sst.isel(latitude=0, longitude=0).plot(ax=ax1)
preprocessed_data.sst.isel(latitude=0, longitude=0).plot(ax=ax2)
fig.tight_layout()
ax1.set_title("Original data")
ax2.set_title("Detrended and climatology removed")

In [ ]:
preprocessed_data.sst.sel(time=slice("1980-08-01", "1980-08-31")).mean(dim="time").plot()